In [ ]:
!nvidia-smi

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.makedirs('/tmp/yolov5',exist_ok=True)

In [ ]:
!git clone https://github.com/kumar-shubham-ml/yolov5
!cp -r yolov5/. /tmp/yolov5/.
!rm -r yolov5

In [ ]:
ls -lh /tmp/yolov5/

## Config

In [ ]:
FOLD = 1
N_CLASSES = 1
IMG_SIZE = 512
BATCH_SIZE = 16
N_EPOCHS = 20
MODEL_NAME = 'yolov5x'
CHECKPOINTS_ROOT = f"{MODEL_NAME}_fold{FOLD}_img{IMG_SIZE}"
EVALUATE = True
RELEASE = None #One of 1-5 (Refer: https://github.com/ultralytics/yolov5/tags)  #### If None, picks the latest version
print(CHECKPOINTS_ROOT)

## Data Config

In [ ]:
%%writefile data.yaml


train: /tmp/images/train
val: /tmp/images/val

# number of classes
nc: 1

# class names
names: ['0']

## Hyperparameters



In [ ]:
%%writefile hyperparameters.yaml

# Hyperparameters for training from scratch

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 3.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 10.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0

## Model Config

In [ ]:
# Read in the model config
try:
    with open(f'/tmp/yolov5/models/{MODEL_NAME}.yaml',) as file :
        filedata = file.read()
except:
    with open(f'/tmp/yolov5/models/hub/{MODEL_NAME}.yaml',) as file :
        filedata = file.read()

# Replace the target string
filedata = filedata.replace('nc: 80', f'nc: {N_CLASSES}' )

# Write the file out again
with open(f'{MODEL_NAME}.yaml','w') as file:
    file.write(filedata)

print(filedata)

## Resolve Requirements Conflicts

In [ ]:
!pip install -r /tmp/yolov5/requirements.txt >/dev/null 2>&1

## Prepare Data

In [ ]:
mask_root = '../input/chest-xray-masks-and-labels/Lung Segmentation/masks/'
img_root = '../input/chest-xray-masks-and-labels/Lung Segmentation/CXR_png/'
all_imgs = os.listdir(img_root)

In [ ]:
os.makedirs('/tmp/images/train/',exist_ok=True)
os.makedirs('/tmp/images/val/',exist_ok=True)
os.makedirs('/tmp/labels/train/',exist_ok=True)
os.makedirs('/tmp/labels/val/',exist_ok=True)

In [ ]:
print(open('../input/siim-lung-boxes-1/CHNCXR_0001_0.txt').read())

In [ ]:
image_file = all_imgs[0]

In [ ]:
import os
from tqdm.auto import tqdm
import joblib
# TRAIN_ROOT_IMAGES = '../input/siim-covid-images-jpegs/images/'
img_root = '/kaggle/input/chest-xray-masks-and-labels/Lung Segmentation/CXR_png/'
mask_root = '/kaggle/input/siim-lung-boxes-1/'
def arrange_images_and_labels(i,image_file):
    text_file = image_file.replace('.png','.txt')
    if i%5==FOLD:
        os.system(f"cp '{img_root}{image_file}' /tmp/images/val/{image_file}")
        os.system(f"cp '{mask_root}{text_file}' /tmp/labels/val/{text_file}")
    else:
        os.system(f"cp '{img_root}{image_file}' /tmp/images/train/{image_file}")
        os.system(f"cp '{mask_root}{text_file}' /tmp/labels/train/{text_file}")
_ = joblib.Parallel(n_jobs=16)(
    joblib.delayed(arrange_images_and_labels)(i,image_file) for i,image_file in tqdm(enumerate(all_imgs))
)

In [ ]:
!ls /tmp/images/train/ | wc -l

In [ ]:
!ls /tmp/labels/train/ | wc -l

In [ ]:
!ls /tmp/images/val/ | wc -l

In [ ]:
!ls /tmp/labels/val/ | wc -l

## Train

In [ ]:
import time
start = time.time()

In [ ]:
# ## Download Model
if RELEASE:
    !wget https://github.com/ultralytics/yolov5/releases/download/v{RELEASE}.0/{MODEL_NAME}.pt

In [ ]:
!WANDB_MODE="dryrun" python3 /tmp/yolov5/train.py --img {IMG_SIZE} --batch {BATCH_SIZE} --epochs {N_EPOCHS} --data data.yaml --cfg {MODEL_NAME}.yaml --name {CHECKPOINTS_ROOT} --weights {MODEL_NAME}.pt --device 0 --hyp hyperparameters.yaml --cache

In [ ]:
print(time.time()-start)

## Visualisation

In [ ]:
## Visualise Results for single batch

import matplotlib.pyplot as plt
print("Targets")
plt.figure(figsize=(30,30))
plt.imshow(plt.imread(f'runs/train/{CHECKPOINTS_ROOT}/test_batch0_labels.jpg'))
plt.show()
print("Prediction")
plt.figure(figsize=(30,30))
plt.imshow(plt.imread(f'runs/train/{CHECKPOINTS_ROOT}/test_batch0_pred.jpg'))
plt.show()

In [ ]:
os.makedirs(CHECKPOINTS_ROOT,exist_ok=True)

In [ ]:
!cp -r runs/train/{CHECKPOINTS_ROOT}/weights {CHECKPOINTS_ROOT}/.
!rm -r runs
!rm -r wandb

In [ ]:
!ls -lh